In [1]:
import sys
import os
import numpy as np
import pathlib
import logging

 
sys.path.append('/gpfs/scratch/pa2297/CCL-Synthetis/')

# import argparse
# parser = argparse.ArgumentParser()

# parser.add_argument('--is_baseline', type=int, default=0, choices=[0,1])
# parser.add_argument('--is_CL', type=int, default=0, choices=[0,1])
# parser.add_argument('--gpus_available', type=str, default='0')
# parser.add_argument('--no_of_tr_imgs', type=str, default='tr10')
# parser.add_argument('--data_aug', type=int, default=1, choices=[0,1])
# # parser.add_argument('--run_num', type=str, default='c3', choices=['c1','c2','c3','c4','c5','c6'])
# parser.add_argument('--lr_finetune', type=float, default=1e-3)
# parser.add_argument('--datatype', type=str, default='t1ce_flair')

# user_cfg = parser.parse_args()

 
import synth_config as cfg

# if not (user_cfg.is_baseline or user_cfg.is_CL):
#     exit('Please select one of the following: baseline or CL pretrained')
# else:
#     print('Starting training')
#     cfg.is_baseline = user_cfg.is_baseline
#     cfg.is_CL = user_cfg.is_CL
#     cfg.gpus_available = user_cfg.gpus_available
#     cfg.no_of_tr_imgs = user_cfg.no_of_tr_imgs
#     cfg.data_aug = user_cfg.data_aug
#     cfg.run_num = user_cfg.run_num
#     cfg.lr_finetune = user_cfg.lr_finetune 

os.environ['CUDA_VISIBLE_DEVICES'] = cfg.gpus_available
# Suppressing TF message printouts
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'  # FATAL
logging.getLogger('tensorflow').setLevel(logging.FATAL)
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

from utils.utils import *
 

opShape = (cfg.img_size_x,cfg.img_size_y) 
cfg.num_channels = len(cfg.contrast_idx)

 
''' Load model architecture '''
from utils.model_utils import modelObj
model = modelObj(cfg)

from synthesis_losses import lossObj
loss = lossObj(cfg)
 


# Load pretrained wts

if cfg.is_baseline:
    print('Baseline model')
    pretrained_model_weights = None
elif cfg.is_CL:
    print('Contrastive loss with CL pretrained model')
    pretrained_model_weights = cfg.CL_param_wts 
else:
    print('Not supported')
   
# Training the model
print('Training RegUNET')
ae_reg = model.synth_unet()
if pretrained_model_weights is not None: 
    print('Loading weights from: ', pretrained_model_weights)

# Choose an appropriate loss function
print('Creating custom loss function for', cfg.loss_function)
if cfg.loss_function == 'L1':
    customLoss = 'mae'
elif cfg.loss_function == 'L2':
    customLoss = 'mse'
else:
    print('Instantiating Vgg16 feature extractor')
    vgg_model = model.vgg16_feature_extractor()
    customLoss = loss.custom_perceptualLoss(vgg_model)

if cfg.optimizer == 'SGD':
    Adamopt = tf.keras.optimizers.SGD(learning_rate=cfg.lr_finetune, momentum=.9) 
else:           
    Adamopt = tf.keras.optimizers.Adam(learning_rate=cfg.lr_finetune) 


ae_reg.compile(loss=customLoss, optimizer=Adamopt)

2024-02-22 15:06:16.662618: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-02-22 15:06:29.968609: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
/gpfs/share/apps/anaconda3/gpu/new/envs/contrastivelearning/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Loss init
Not supported
Training RegUNET


NameError: name 'pretrained_model_weights' is not defined

In [2]:
cfg.CL_param_wts 

'/gpfs/scratch/pa2297/pretraining/Test-2/pretrain_full_dec_zmean/t1_t2_cd_flair/patchsize_4_LR0.001_tau0.1_top100/weights_150.hdf5'

In [3]:
pretrained_model_weights = cfg.CL_param_wts 

In [4]:
# Training the model
print('Training RegUNET')
ae_reg = model.synth_unet()
if pretrained_model_weights is not None: 
    print('Loading weights from: ', pretrained_model_weights)

# Choose an appropriate loss function
print('Creating custom loss function for', cfg.loss_function)
if cfg.loss_function == 'L1':
    customLoss = 'mae'
elif cfg.loss_function == 'L2':
    customLoss = 'mse'
else:
    print('Instantiating Vgg16 feature extractor')
    vgg_model = model.vgg16_feature_extractor()
    customLoss = loss.custom_perceptualLoss(vgg_model)

if cfg.optimizer == 'SGD':
    Adamopt = tf.keras.optimizers.SGD(learning_rate=cfg.lr_finetune, momentum=.9) 
else:           
    Adamopt = tf.keras.optimizers.Adam(learning_rate=cfg.lr_finetune) 


ae_reg.compile(loss=customLoss, optimizer=Adamopt)

Training RegUNET
Loading weights from:  /gpfs/scratch/pa2297/pretraining/Test-2/pretrain_full_dec_zmean/t1_t2_cd_flair/patchsize_4_LR0.001_tau0.1_top100/weights_150.hdf5
Creating custom loss function for hybrid
Instantiating Vgg16 feature extractor


AttributeError: 'modelObj' object has no attribute 'vgg16_feature_extractor'

In [5]:
cfg.loss_function = 'L1'

In [6]:
# Choose an appropriate loss function
print('Creating custom loss function for', cfg.loss_function)
if cfg.loss_function == 'L1':
    customLoss = 'mae'
elif cfg.loss_function == 'L2':
    customLoss = 'mse'
else:
    print('Instantiating Vgg16 feature extractor')
    vgg_model = model.vgg16_feature_extractor()
    customLoss = loss.custom_perceptualLoss(vgg_model)

if cfg.optimizer == 'SGD':
    Adamopt = tf.keras.optimizers.SGD(learning_rate=cfg.lr_finetune, momentum=.9) 
else:           
    Adamopt = tf.keras.optimizers.Adam(learning_rate=cfg.lr_finetune) 


ae_reg.compile(loss=customLoss, optimizer=Adamopt)

Creating custom loss function for L1


In [7]:
from Datagen.h5_pretrain_Synth_Data_Generator import DataLoaderObj

train_gen = DataLoaderObj(cfg, 
                        train_flag=True) 

val_gen = DataLoaderObj(cfg, 
                        train_flag=False)

Initializing training dataloader
Total samples : 4000
Initializing validation dataloader
Total samples : 2400


In [8]:
X,Y = next(iter(train_gen))

In [9]:
ae_reg.train_on_batch(X,Y)

1.8777832984924316

In [10]:
ae_reg.train_on_batch(X,Y)

1.37422513961792

In [11]:
ae_reg.train_on_batch(X,Y)

0.9458797574043274